### Time Series Forcasting with Long Short-Term Memory 2

In this notebook I will investigate:
- univariate time series forcasting
- multivariate time series forcasting
- multi-step time series forcasting

time series forcasting.

This notebook is based on the tutorial by Jason Brownlee and here is the link to the tutorial
https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/


## Multivariate LSTM Models


In the case of multivariate time series, it means that we have more than one observation for each time step.

The problem can have Multiple Input Series or Multiple Parallel Series.

#### Multiple Input Series